In [2]:
import os
from openai import OpenAI

first, I installed tiktoken

In [3]:
import tiktoken

choose encoding model:

- o200k_base: GPT-4o-Mini model.
- cl100k_base: GPT-4 and GPT-3.5-Turbo.
- p50k_base: Encoding for Codex models, these models are used for code applications.
- r50k_base: Older encoding for different versions of GPT-3.

In [4]:
# How to check encoding used in a particular model?

print(tiktoken.encoding_for_model('text-embedding-3-small'))
print(tiktoken.encoding_for_model('gpt-3.5-turbo'))

print(tiktoken.encoding_for_model('gpt-4o-mini'))

<Encoding 'cl100k_base'>
<Encoding 'cl100k_base'>
<Encoding 'o200k_base'>


In [5]:
tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [6]:
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')

# or equivalently:
# encoding = tiktoken.get_encoding('cl100k_base')

Now, try out this encoding

In [7]:
text_to_test = 'Short sentence to get the tokens'
tokens_result = encoding.encode(text_to_test)
print(tokens_result)
print(f'Number of tokens used is: {len(tokens_result)}.')

[12755, 11914, 311, 636, 279, 11460]
Number of tokens used is: 6.


For completeness, get the decoding:

In [8]:
print(f'From the tokens: {tokens_result} we get the following text:\n{encoding.decode(tokens_result)}')

From the tokens: [12755, 11914, 311, 636, 279, 11460] we get the following text:
Short sentence to get the tokens


try the library openai-cost-tracker

In [9]:
from openai_cost_tracker import query_openai

In [10]:
# my key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [10]:
import openai 
from openai_cost_tracker import query_openai

openai.api_key = "OPENAI_API_KEY"

prompt = "Hello World!"  # your prompt here

response = query_openai(
    model="gpt-4-0125-preview",  # support gpt-4-0125-preview,  gpt-3.5-turbo-1106,  gpt-4
    messages=[{'role': 'user', 'content': prompt}],            
    max_tokens=5,
    # rest of your OpenAI params here ...
    simulation=True,  # set to True to test the cost of a request without actually sending it to OpenAI 
    print_cost=True   # set to True to print the cost of each request
)     

print(response["choices"][0]["message"]["content"])

Input tokens: 10 | Output tokens: 2 | Cost: $0.0002 | Total: $0.0027
reiciendis voluptates


try with one of my actual calls from hs7_fn

In [1]:
from openai import OpenAI
import os
from openai_cost_tracker import query_openai
from utilities import classify_hs_cost_test

client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

user_input = "I would kill for a burger!"
protected_characteristics = [
    'race', 'religion', 'ethnic origin', 'national origin', 'sex', 'disability', 'sexual orientation', 'gender identity'
]
# DEFINITION OF HATE SPEECH:
HS_definition = '"Hate speech" is speech that attacks a person or group on the basis of attributes such as: ' + ', '.join(protected_characteristics) +'.'

print(classify_hs_cost_test(
    message = user_input,
    protected_characteristics_str = ", ".join(protected_characteristics),
    HS_definition=HS_definition, 
    examples=[],
    # examples=hs_examples_str,
    chain_ot=[],
    # chain_ot=chain_ot_str,
    verbose = True,
    # context=user_input_context
    ))


    DEFINITIONS:
    Consider the following definition: "Hate speech" is speech that attacks a person or group on the basis of attributes such as: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity.. 
    
    CONTEXT:
    Consider the following CONTEXT:

    INSTRUCTION: 
    Using the provided definition of hate speech, classify the following fragment from a chat as either hate speech with respect to one or more of protected characteristics from the following list: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity, or not hate speech with respect to the protected characteristics from the following list: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity.
    

    OUTPUT:
    The output should only contain 3 elements: 
    1) "hate speech" or "not hate speech", 
    2) list of protected characteristic labels from the list: race, relig

here is a section with 

- protected characteristics
- definition of hate speech
- examples
- chain of thought - the standard that is used now


In [4]:
############################################################
protected_characteristics = [
    'race', 'religion', 'ethnic origin', 'national origin', 'sex', 'disability', 'sexual orientation', 'gender identity'
]
# DEFINITION OF HATE SPEECH:
HS_definition = '"Hate speech" is speech that attacks a person or group on the basis of attributes such as: ' + ', '.join(protected_characteristics) +'.'

############################################################
# EXAMPLES!!!
hs_examples_dict = [
    {'example input':'You lost! Succer!',
     'example output':['not hate speech',[''], [1]]},
    {'example input':'you hateful sack of flesh',
     'example output':['not hate speech',[''], [1]]},
    {'example input':'“Pick one, kidd,” one nudged. Then, “Choose FA GO TT.”',
     'example output':['hate speech',['sexual orientation'], [1]]},
    # {'example input':'Aspie my ass. too slow to remember, so dont waste our time playing',
    #  'example output':['hate speech',['neurotype'], [1]]},
    {'example input':'“Kept being called a "Jew" for no reason.”; "A player on the other team was named Hitler.” “[One player] threatened to get antisemitic if I didnt talk. [They] talked about Jew tunnels.”',
     'example output':['hate speech',['race'], [1]]},
    {'example input':'if money was grown on trees women would be dating monkeys oh wait never mind',
     'example output':['hate speech',['sex'], [1]]},
    {'example input':'well my dear lgbtq brothers and sisters i do not want to die at the hands of a muslim terrorist and as you are putting my life in danger with your politics i would like to suggest that you all go fuck yourselves',
     'example output':['hate speech',['gender identity', 'sexual orientation', 'religion'], [0.97,0.97,1]]}
]


hs_examples_str = '\n'
for ind in range(len(hs_examples_dict)):
  hs_examples_str += "\n" + str(hs_examples_dict[ind])[1:-1]
############################################################

# CHAIN OF THOUGHT EXAMPLES!!!
chain_of_thought_dict = [
    {"example input":"There are Jews and Muslims here",
     "example output":['not hate speech',[''], [1]],
     "explanation":"This is a statement naming people's religions. It is neutral in tone, regadless if it's factually correct or not. So it is not hate speech"},
    {"example input":"There are Catholics and Muslims here. Let's watch as they k*ll each other.",
     "example output":['hate speech',['religion'], [1]],
     "explanation":"A neutral statement naming people's religions is followed by an implied suggestion that having those religious beliefs will cause people to fight each other. So it is hate speech"},
    {"example input":"You're saying you're a Sikh? I say turban wearing bin laden",
     "example output":['hate speech',['religion'], [1]],
     "explanation":"Person's religion (Sikhism) is devalued and their religious clothing (turban) is compared to a symbol of terrorism - Osama bin Laned. So it is hate speech"},
     {"example input":"Queers are welcome here",
     "example output":['not hate speech',['sexual orientation','gender identity'], [1]],
     "explanation":"Statement implies inclusion and acceptance. So it is not hate speech"},
     {"example input":"All those alphabet people should f**k off from here and stop grooming children",
     "example output":['hate speech',['sexual orientation','gender identity'], [1]],
     "explanation":"'alphabet people' is a slang term referring to LGBTQ+ community and is used in a hostile context. Suggestion about hurt children is used to incite hatred against LGBTQ+ people. So it is hate speech"},
]


chain_ot_str = '\n'
for ind in range(len(chain_of_thought_dict)):
  chain_ot_str += "\n" + str(chain_of_thought_dict[ind])[1:-1]
############################################################

In [5]:
from openai import OpenAI
import os
from openai_cost_tracker import query_openai
from utilities import classify_hs_cost_test

client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

user_input = "I would kill for a burger!"

print(classify_hs_cost_test(
    message = user_input,
    protected_characteristics_str = ", ".join(protected_characteristics),
    HS_definition=HS_definition, 
    examples=[],
    # examples=hs_examples_str,
    # chain_ot=[],
    chain_ot=chain_ot_str,
    verbose = True,
    # context=user_input_context
    ))


    DEFINITIONS:
    Consider the following definition: "Hate speech" is speech that attacks a person or group on the basis of attributes such as: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity.. 
    
    CONTEXT:
    Consider the following CONTEXT:

    INSTRUCTION: 
    Using the provided definition of hate speech, classify the following fragment from a chat as either hate speech with respect to one or more of protected characteristics from the following list: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity, or not hate speech with respect to the protected characteristics from the following list: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity.
    

    OUTPUT:
    The output should only contain 3 elements: 
    1) "hate speech" or "not hate speech", 
    2) list of protected characteristic labels from the list: race, relig

In [6]:
from openai import OpenAI
import os
from openai_cost_tracker import query_openai
from utilities import classify_hs_cost_test

client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

user_input = "I would kill for a burger!"


print(classify_hs_cost_test(
    message = user_input,
    protected_characteristics_str = ", ".join(protected_characteristics),
    HS_definition=HS_definition, 
    # examples=[],
    examples=hs_examples_str,
    # chain_ot=[],
    chain_ot=chain_ot_str,
    verbose = True,
    # context=user_input_context
    ))


    DEFINITIONS:
    Consider the following definition: "Hate speech" is speech that attacks a person or group on the basis of attributes such as: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity.. 
    
    CONTEXT:
    Consider the following CONTEXT:

    INSTRUCTION: 
    Using the provided definition of hate speech, classify the following fragment from a chat as either hate speech with respect to one or more of protected characteristics from the following list: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity, or not hate speech with respect to the protected characteristics from the following list: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity.
    

    OUTPUT:
    The output should only contain 3 elements: 
    1) "hate speech" or "not hate speech", 
    2) list of protected characteristic labels from the list: race, relig

In [14]:
aaa = "ab,2"
aaa.split

''

Now with an actual hateful text to analise

In [7]:
from openai import OpenAI
import os
from openai_cost_tracker import query_openai
from utilities import classify_hs_cost_test

client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

user_input = "I would kill for a black burger!"


print(classify_hs_cost_test(
    message = user_input,
    protected_characteristics_str = ", ".join(protected_characteristics),
    HS_definition=HS_definition, 
    # examples=[],
    examples=hs_examples_str,
    # chain_ot=[],
    chain_ot=chain_ot_str,
    verbose = True,
    # context=user_input_context
    ))


    DEFINITIONS:
    Consider the following definition: "Hate speech" is speech that attacks a person or group on the basis of attributes such as: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity.. 
    
    CONTEXT:
    Consider the following CONTEXT:

    INSTRUCTION: 
    Using the provided definition of hate speech, classify the following fragment from a chat as either hate speech with respect to one or more of protected characteristics from the following list: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity, or not hate speech with respect to the protected characteristics from the following list: race, religion, ethnic origin, national origin, sex, disability, sexual orientation, gender identity.
    

    OUTPUT:
    The output should only contain 3 elements: 
    1) "hate speech" or "not hate speech", 
    2) list of protected characteristic labels from the list: race, relig

and a short test for inbuilt moderation endpoint! just because!

from: https://platform.openai.com/docs/guides/moderation/overview?moderation-quickstart-examples=text#content-classifications

In [16]:
response_mod = client.moderations.create(
  model="omni-moderation-latest",
  input="killing time in the name of thyme",
)

print(response_mod.results[0])
print(response_mod.results[0].flagged)

response_mod = client.moderations.create(
  model="omni-moderation-latest",
  input="killing in the name of clean, superior race",
)

print(response_mod.results[0])
print(response_mod.results[0].flagged)

Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=False, illicit_violent=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, harassment/threatening=False, hate/threatening=False, illicit/violent=False, self-harm/intent=False, self-harm/instructions=False, self-harm=False, sexual/minors=False, violence/graphic=False), category_applied_input_types=CategoryAppliedInputTypes(harassment=['text'], harassment_threatening=['text'], hate=['text'], hate_threatening=['text'], illicit=['text'], illicit_violent=['text'], self_harm=['text'], self_harm_instructions=['text'], self_harm_intent=['text'], sexual=['text'], sexual_minors=['text'], violence=['text'], violence_graphic=['text'], harassment/threatening=['text'], hate/threatening=['text'], illicit/violent=['text'], self-harm/intent=['text'], self-harm/instructions=['text'

In [21]:
import pandas as pd
aa = [{'a':1, 'b':2, 'c':'3'}, {'a':11, 'b':22, 'c':'33'}]
print(aa)
adf = pd.DataFrame(aa)
print(adf)

print('Apace!\n')
print(adf[adf['a']==1])

[{'a': 1, 'b': 2, 'c': '3'}, {'a': 11, 'b': 22, 'c': '33'}]
    a   b   c
0   1   2   3
1  11  22  33
Apace!

   a  b  c
0  1  2  3


In [22]:
a = ';kljagfdkhzfdl;j'
b = a[:6]
print(a)
print(b)

;kljagfdkhzfdl;j
;kljag


In [23]:
txt = "Tesla CEO Elon Musk has revealed that he was tricked into giving consent for his child to go on puberty blockers, adding that 'the woke mind virus' figuratively killed his elder son. \nIn an interview with Canadian psychologist and author Jordan Peterson, Musk spoke about his concerns regarding children undergoing procedures related to gender dysphoria—the clinical diagnosis for significant distress resulting from an incongruence between a person’s gender identity and the sex they were assigned at birth. \nHis comments came two years after reports emerged that his son Xavier Alexander Musk decided to transition into a woman. In 2022, Xavier turned into Vivian Jenna Wilson and dropped the surname 'Musk' to sever ties with the Tesla and SpaceX founder, according to US media reports. \n‘Tricked into signing documents’\nDuring a live stream on social media platform X, Musk described the term 'gender-affirming care' as a misleading euphemism. Musk, who is one of the richest men on Earth, described these procedures as unethical practices that lead to sterilisation and irreversible changes among teens. \n'I was essentially tricked into signing documents for one of my older boys,' Musk revealed, reflecting on his experience with his son Xavier’s transition. 'I lost my son, essentially,' he said, referring to the concept of ‘deadnaming’ as the loss of a child due to what he termed the 'woke mind virus.' \nPeterson, known for his critique of political correctness and social justice movements, supported Musk’s views. 'No reliable clinician ever believed that,' Peterson stated regarding claims of increased suicide risk among transgender individuals due to gender dysphoria. Musk further slammed the promotion of these procedures, suggesting those advocating for them should face legal consequences. \n'It’s incredibly evil,' he said, advocating for stricter oversight and education on the implications of such medical decisions, particularly for minors. The discussion between Musk and Peterson comes amid ongoing debate on the ethical considerations of medical interventions for minors struggling with gender dysphoria. \nIn the United States, Republican-led states have passed several laws in recent years targeting medications or surgical interventions for adolescents with gender dysphoria. The law bans healthcare workers from administering puberty blockers and hormones for purposes 'inconsistent with the minor’s sex.' Providers can be sued and face fines and professional discipline for violations."

print(txt)

Tesla CEO Elon Musk has revealed that he was tricked into giving consent for his child to go on puberty blockers, adding that 'the woke mind virus' figuratively killed his elder son. 
In an interview with Canadian psychologist and author Jordan Peterson, Musk spoke about his concerns regarding children undergoing procedures related to gender dysphoria—the clinical diagnosis for significant distress resulting from an incongruence between a person’s gender identity and the sex they were assigned at birth. 
His comments came two years after reports emerged that his son Xavier Alexander Musk decided to transition into a woman. In 2022, Xavier turned into Vivian Jenna Wilson and dropped the surname 'Musk' to sever ties with the Tesla and SpaceX founder, according to US media reports. 
‘Tricked into signing documents’
During a live stream on social media platform X, Musk described the term 'gender-affirming care' as a misleading euphemism. Musk, who is one of the richest men on Earth, descri

In [24]:
msgs = [
  'I would kill for a burger',
  "Hey dude, just because you think you are a woman, doesn't mean we have to accept it. Male and female is not a belief, its just reality.",
  "Tesla CEO Elon Musk has revealed that he was tricked into giving consent for his child to go on puberty blockers, adding that 'the woke mind virus' figuratively killed his elder son. \nIn an interview with Canadian psychologist and author Jordan Peterson, Musk spoke about his concerns regarding children undergoing procedures related to gender dysphoria—the clinical diagnosis for significant distress resulting from an incongruence between a person’s gender identity and the sex they were assigned at birth. \nHis comments came two years after reports emerged that his son Xavier Alexander Musk decided to transition into a woman. In 2022, Xavier turned into Vivian Jenna Wilson and dropped the surname 'Musk' to sever ties with the Tesla and SpaceX founder, according to US media reports. \n‘Tricked into signing documents’\nDuring a live stream on social media platform X, Musk described the term 'gender-affirming care' as a misleading euphemism. Musk, who is one of the richest men on Earth, described these procedures as unethical practices that lead to sterilisation and irreversible changes among teens. \n'I was essentially tricked into signing documents for one of my older boys,' Musk revealed, reflecting on his experience with his son Xavier’s transition. 'I lost my son, essentially,' he said, referring to the concept of ‘deadnaming’ as the loss of a child due to what he termed the 'woke mind virus.' \nPeterson, known for his critique of political correctness and social justice movements, supported Musk’s views. 'No reliable clinician ever believed that,' Peterson stated regarding claims of increased suicide risk among transgender individuals due to gender dysphoria. Musk further slammed the promotion of these procedures, suggesting those advocating for them should face legal consequences. \n'It’s incredibly evil,' he said, advocating for stricter oversight and education on the implications of such medical decisions, particularly for minors. The discussion between Musk and Peterson comes amid ongoing debate on the ethical considerations of medical interventions for minors struggling with gender dysphoria. \nIn the United States, Republican-led states have passed several laws in recent years targeting medications or surgical interventions for adolescents with gender dysphoria. The law bans healthcare workers from administering puberty blockers and hormones for purposes 'inconsistent with the minor’s sex.' Providers can be sued and face fines and professional discipline for violations."
]

for msg in msgs:
    print(msg)

I would kill for a burger
Hey dude, just because you think you are a woman, doesn't mean we have to accept it. Male and female is not a belief, its just reality.
Tesla CEO Elon Musk has revealed that he was tricked into giving consent for his child to go on puberty blockers, adding that 'the woke mind virus' figuratively killed his elder son. 
In an interview with Canadian psychologist and author Jordan Peterson, Musk spoke about his concerns regarding children undergoing procedures related to gender dysphoria—the clinical diagnosis for significant distress resulting from an incongruence between a person’s gender identity and the sex they were assigned at birth. 
His comments came two years after reports emerged that his son Xavier Alexander Musk decided to transition into a woman. In 2022, Xavier turned into Vivian Jenna Wilson and dropped the surname 'Musk' to sever ties with the Tesla and SpaceX founder, according to US media reports. 
‘Tricked into signing documents’
During a live 

In [25]:
len(msgs)

3

In [34]:
#what's wrong with my lists/dicts/data frames?

aaaa = [{'Text': 'I would kill for a b', 'Model': 'gpt-4o-mini-2024-07-18', 'Includes': ',', 'Input Tokens': 309, 'Output Tokens': 9, 'Total cost': 5.175e-05, 'Response': "['not hate speech', [], []]"}, {'Text': 'I would kill for a b', 'Model': 'gpt-4o-mini-2024-07-18', 'Includes': 'hs_examples_str,chain_ot_str', 'Input Tokens': 315, 'Output Tokens': 9, 'Total cost': 5.2650000000000006e-05, 'Response': "['not hate speech', [], []]"}]
len(aaaa)

import pandas as pd

# display the results in a table format

df = pd.DataFrame(aaaa)
print(df)

                   Text                   Model                      Includes  \
0  I would kill for a b  gpt-4o-mini-2024-07-18                             ,   
1  I would kill for a b  gpt-4o-mini-2024-07-18  hs_examples_str,chain_ot_str   

   Input Tokens  Output Tokens  Total cost                     Response  
0           309              9    0.000052  ['not hate speech', [], []]  
1           315              9    0.000053  ['not hate speech', [], []]  


In [36]:
with open('clarkson.txt', 'r') as file:
    data = file.read().rstrip()

print(data)
print(type(data))
len(data)

Clarkson defends Meghan
In a Sun column written in December 2018, Clarkson said Meghan had been "a breath of fresh air for the royals", but went on to say she had become "the wicked witch". Clarkson actually went on to defend Meghan, saying he "doesn't know her" and couldn't understand why she was being mocked.

"As a result, she will have less impact on your life than your postman's wife," he wrote. "It's ridiculous, cruel and heartless — and unless she proves me wrong by going around at night urinating on homeless people, it needs to stop." When Meghan and Harry announced their decision to stop being working royals, Clarkson said it may have come as a surprise to Meghan that life as a British royal and it wasn't all "fairy-tale castles" and "riding around on golden unicorns". But he said he felt it was "unfair" to blame her, writing: "I won't judge her for that. People are allowed to resign from jobs they don't like."

The nasty obsession begins
It didn't last, as Clarkson went back 

4216

In [37]:
data[-1]

'"'

In [42]:
msgs = [
  'I would kill for a burger',
  "Hey dude, just because you think you are a woman, doesn't mean we have to accept it. Male and female is not a belief, its just reality.",
  "Tesla CEO Elon Musk has revealed that he was tricked into giving consent for his child to go on puberty blockers, adding that 'the woke mind virus' figuratively killed his elder son. \nIn an interview with Canadian psychologist and author Jordan Peterson, Musk spoke about his concerns regarding children undergoing procedures related to gender dysphoria—the clinical diagnosis for significant distress resulting from an incongruence between a person’s gender identity and the sex they were assigned at birth. \nHis comments came two years after reports emerged that his son Xavier Alexander Musk decided to transition into a woman. In 2022, Xavier turned into Vivian Jenna Wilson and dropped the surname 'Musk' to sever ties with the Tesla and SpaceX founder, according to US media reports. \n‘Tricked into signing documents’\nDuring a live stream on social media platform X, Musk described the term 'gender-affirming care' as a misleading euphemism. Musk, who is one of the richest men on Earth, described these procedures as unethical practices that lead to sterilisation and irreversible changes among teens. \n'I was essentially tricked into signing documents for one of my older boys,' Musk revealed, reflecting on his experience with his son Xavier’s transition. 'I lost my son, essentially,' he said, referring to the concept of ‘deadnaming’ as the loss of a child due to what he termed the 'woke mind virus.' \nPeterson, known for his critique of political correctness and social justice movements, supported Musk’s views. 'No reliable clinician ever believed that,' Peterson stated regarding claims of increased suicide risk among transgender individuals due to gender dysphoria. Musk further slammed the promotion of these procedures, suggesting those advocating for them should face legal consequences. \n'It’s incredibly evil,' he said, advocating for stricter oversight and education on the implications of such medical decisions, particularly for minors. The discussion between Musk and Peterson comes amid ongoing debate on the ethical considerations of medical interventions for minors struggling with gender dysphoria. \nIn the United States, Republican-led states have passed several laws in recent years targeting medications or surgical interventions for adolescents with gender dysphoria. The law bans healthcare workers from administering puberty blockers and hormones for purposes 'inconsistent with the minor’s sex.' Providers can be sued and face fines and professional discipline for violations."
]

print(len(msgs))

msgs.append(data)

msgs.append(data)

len(msgs)

3


5

In [43]:
print(msgs[2])

Tesla CEO Elon Musk has revealed that he was tricked into giving consent for his child to go on puberty blockers, adding that 'the woke mind virus' figuratively killed his elder son. 
In an interview with Canadian psychologist and author Jordan Peterson, Musk spoke about his concerns regarding children undergoing procedures related to gender dysphoria—the clinical diagnosis for significant distress resulting from an incongruence between a person’s gender identity and the sex they were assigned at birth. 
His comments came two years after reports emerged that his son Xavier Alexander Musk decided to transition into a woman. In 2022, Xavier turned into Vivian Jenna Wilson and dropped the surname 'Musk' to sever ties with the Tesla and SpaceX founder, according to US media reports. 
‘Tricked into signing documents’
During a live stream on social media platform X, Musk described the term 'gender-affirming care' as a misleading euphemism. Musk, who is one of the richest men on Earth, descri